# Exercícios

Sistemas de Recomendação
Para esse conjunto de exercícios vamos utilizar novamente o dataset MovieLens 100k que utilizamos em nosso Warmup

Utilizando esse Dataset, implemente as funcionalidades a seguir, podendo agora utilizar as bibliotecas padrões do python (numpy, math, scipy ) para implementar suas soluções. Faça a organização do seu código agora utilizando funções buscando reutilizar seu código.

In [1]:
from datetime import datetime
import math
import random
import numpy as np
import copy

# Função para leitura do arquivo, a partir do caminho ele salvo na variavel arquivo ele faz a leitura das linhas e salva dentro de um array final de forma estruturada
def get_data(arquivo, split, append):
    final = []

    with open('ml-100k/' + arquivo, encoding="ISO-8859-1") as arq:
        for lines in arq:
            arr=lines.split(split)
            if(len(arr)>1):
                final.append(append(arr))
    return final

In [2]:
# De maneira organizada ele vai guardar os valores em um dicionario, usando o rotulo e a posição no array para valor do mesmo
append_review = lambda arr : {
            "user": int(arr[0]),
            "movie": int(arr[1]),
            "rating": int(arr[2]),
            "year": datetime.fromtimestamp(int(arr[3])).year
        }
        
movies = get_data('u.item', '|', lambda arr: {
            "id": int(arr[0]),
            "movie":        arr[1],
            "unknown":      int(arr[5]),
            "Action":       int(arr[6]),
            "Adventure":    int(arr[7]),
            "Animation":    int(arr[8]),
            "Children's":   int(arr[9]),
            "Comedy":       int(arr[10]),
            "Crime":        int(arr[11]),
            "Documentary":  int(arr[12]),
            "Drama":        int(arr[13]),
            "Fantasy":      int(arr[14]),
            "Film-Noir":    int(arr[15]),
            "Horror":       int(arr[16]),
            "Musical":      int(arr[17]),
            "Mystery":      int(arr[18]),
            "Romance":      int(arr[19]),
            "Sci-Fi":       int(arr[20]),
            "Thriller":     int(arr[21]),
            "War":          int(arr[22]),
            "Western":      int(arr[23])
        })

reviews = get_data('u.data', None, lambda arr : {
            "user": int(arr[0]),
            "movie": int(arr[1]),
            "rating": int(arr[2]),
            "year": datetime.fromtimestamp(int(arr[3])).year
        })

1. Implementar o algoritmo de Similaridade do Cosseno;
- A similaridade por cosseno é uma medida da similareda de entre dois vetores num espaço vetorial que avalia o valor do cosseno do ângulo compreendido entre eles. 

In [3]:
# Função criada com entrada para duas variaveis, também criada a variavel para soma final e duas auxiliares. Primeiramente ele faz um comparativo dos valores das duas
# variaveis, se caso corresponder a condição ele fara o calculo da similiaridade do cosseno -> A x B / ||A|| x ||B||
def cosseno(V1, V2):
    sum = 0
    auxA = 0
    auxB = 0
    if(len(V1) == len(V2)):
        for i in range(len(V1)):
            sum = sum + V1[i]*V2[i]
            auxA = auxA + (V1[i])**2
            auxB = auxB + (V2[i])**2
        cosine = sum/(math.sqrt(auxA)*math.sqrt(auxB))
        return cosine

# Cria variaveis e atribui um valor random que corresponde a um filme
film1genres = list(movies[random.randrange(len(movies))].values())[2:]
film2genres = list(movies[random.randrange(len(movies))].values())[2:]

# Pega os 2 filmes aleatorios, e calcula a similaridade do cosseno a partir dos seus generos
cosseno(film1genres, film2genres)

0.7071067811865475

2. Implementar um Sistema de Recomendações, usando a técnica de Filtros Colaborativos baseados no usuário;

In [4]:
# Numero de indicacoes
number_of_indications=5
# Usuario aletorio
user_number= random.randint(1, 943)
# vetor de generos cru, onde o review positivos serão somados
genres_vector = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# armazena filmes que o usuario deu review
user_movies = []
# ranking de filmes selecionados
top = []

# faz leitura de todos filmes que usuario deu review e somar o vetor de generos
for i in reviews:
    if(i['user'] == user_number):
        user_movies.append(i["movie"])
        if(i['rating'] >= 4):
            genres = np.array(list(movies[i['movie']].values())[2:])
            genres_vector = np.add(np.array(genres_vector), genres)

# faz loop no filme, verifica se o filme esta nos filmes que usuario deu review, salva filme no array top
for i in movies:
    if not (i['id'] in user_movies):
        genres = list(i.values())[2:]
        new = copy.deepcopy(i)
        new['cos'] = cosseno(genres_vector, genres)
        top.append(new)

# Ordena por valor de cos
sorted_arr = sorted(top, key=lambda d: d['cos'], reverse=True)[:number_of_indications]

# Apresenta os filmes
print(f'Baseado nos reviews do usuario {user_number}, talvez ele goste de: ')
for idx, i in enumerate(sorted_arr):
    print(f'{idx+1} - {i["movie"]}')

Baseado nos reviews do usuario 754, talvez ele goste de: 
1 - Cinema Paradiso (1988)
2 - Wings of Desire (1987)
3 - Manhattan (1979)
4 - American President, The (1995)
5 - Corrina, Corrina (1994)


3.Implementar um Sistema de Recomendações, usando a técnica de Filtros Colaborativos baseados na similaridade entre os itens (filmes);

In [8]:
# numero de filmes que serão apresentados
number_of_indications=10
# ranking de filmes selecionados
top = []
# numero aleatorio de filmes, variação do numero 2 ate o numero total de filmes
movie_number = random.randint(1, len(movies)+1)
# lista de filmes, pega o index do filme selecionado
get_movie = movies[movie_number-1]
# transforma dicionario de filme para uma lista e a partir dele ele pega os dois valores para frente
film_genres = list(get_movie.values())[2:]

# Verifica em todos os filmes
for i in movies:
    if(i["id"] != movie_number):
        genres = list(i.values())[2:] # pega os generos dentro do loop, a partir da 2 (id, nome, generos)
        new = copy.deepcopy(i) # copia profunda, onde não altera o valor de origem
        new['cos'] = cosseno(film_genres, genres) # faz verificacao do cosseno para verificar a semelhança 
        top.append(new) # sobe no array o filme que se assemelha, ao final lista todos os filmes semelhantes

# ordena o array com base no valor de cos, pega os 10 primeiros valores 
sorted_arr = sorted(top, key=lambda d: d['cos'], reverse=True)[:number_of_indications]

# printa os filmes
print(f'Filmes similares a {get_movie["movie"]}: ')

# apresenta os filmes, utiliza idx para pegar os dados do filme pelo index
for idx, i in enumerate(sorted_arr):
    print(f'{idx+1} - {i["movie"]}')

Filmes similares a Inventing the Abbotts (1997): 
1 - Postino, Il (1994)
2 - Angels and Insects (1995)
3 - Mad Love (1995)
4 - Phenomenon (1996)
5 - Breakfast at Tiffany's (1961)
6 - Graduate, The (1967)
7 - Room with a View, A (1986)
8 - Jerry Maguire (1996)
9 - Chasing Amy (1997)
10 - Pillow Book, The (1995)


4.Implementar a Medida de Correlação de Pearson como medida de similaridade;

In [6]:
from scipy.stats import pearsonr

x = pearsonr([-2, -1, 0, 1, 2], [4, 1, 3, 2, 0])

print(x.pvalue)

0.1881204043741873
